# Is park weather associated with visitor counts for 2022?

### Hypothesis

Null:  Weather and visitation are not correlated.

Alternate:  Parks with average summer temperatures between 70-80 degrees have more visitors.

In [ ]:
#Dependencies
import pandas as pd
import json
import requests
import csv
import matplotlib.pyplot as plt
import scipy.stats as st
from configure import VC_API_key

In [ ]:
# Pull visitor and park info from resources, especially looking for lat and long and at the weather column.

# Dan's code
from configure import NPS_API_key
from pathlib import Path
import re
from pandas import json_normalize 

park_visitors_csv_file = Path("../Resources/national_park_visitors.csv")

#Read the csv and create variable called park_visitors_data
park_visitors_data= pd.read_csv(park_visitors_csv_file)

#Make it into a dataframe
park_visitors_df = pd.DataFrame(park_visitors_data)

#Establish nps endpoint - with api key
endpoint = f"https://developer.nps.gov/api/v1/parks?limit=600&api_key={NPS_API_key}"

#turn the result into a json file
parks_data = requests.get(endpoint).json()

#Normalize data so it all fits into a dataframe
normalized_parks_data = json_normalize(parks_data, 'data')

#Drop the columns we dont need
all_parks_df = pd.DataFrame(normalized_parks_data).drop(columns=[
'id', 
    'url', 
    'directionsInfo', 
    'directionsUrl', 
    'addresses',
    'latLong',
    'images', 
    'contacts.phoneNumbers',
    'contacts.emailAddresses',
    
])

#sort by only those named national park or national park and preserve
national_parks_df = all_parks_df.loc[(all_parks_df['designation'] == "National Park") | (all_parks_df['designation'] == "National Park & Preserve")]

#Clean up the " NP" Part of the Park Name
park_visitors_df_NP_clean = park_visitors_df.replace(' NP','', regex=True)

#Clean up the "& PRES" Part of the Park Name
park_visitors_df_PRES_clean = park_visitors_df_NP_clean.replace(' & PRES','', regex=True)

#Make Park Name match for those that do not
park_visitors_df_clean = park_visitors_df_PRES_clean.replace('Hawaii Volcanoes','Hawaiʻi Volcanoes', regex=True)
park_visitors_df_clean = park_visitors_df_clean.replace('Haleakala','Haleakalā', regex=True)
park_visitors_df_clean = park_visitors_df_clean.replace('Black Canyon of the Gunnison','Black Canyon Of The Gunnison', regex=True)
park_visitors_df_clean = park_visitors_df_clean.replace('Wrangell-St. Elias','Wrangell - St Elias', regex=True)
park_visitors_df_clean = park_visitors_df_clean.replace('Gates of the Arctic','Gates Of The Arctic', regex=True)

#rename park_visitors_df columns
park_visitors_df_clean.columns = ["Park Name", "Rank", "Visitors", "Percent Of Total"]

#create a copy to start renaming columns
national_parks_df_clean = national_parks_df.copy()

#rename columns
national_parks_df_clean.rename(columns={
    'fullName':'Full Name',
    'parkCode':'Park Code',
    'description':'Description',
    'latitude':'Latitude',
    'longitude':'Longitude',
    'activities':'Activities',
    'topics':'Topics',
    'states':'States',
    'entranceFees':'Entrance Fees',
    'entrancePasses':'Entrance Passes',
    'fees':'Fees',
    'operatingHours':'Operating Hours',
    'weatherInfo':'Weather Info',
    'designation':'Designation',
    'name':'Park Name'
}, inplace = True)

#merge visitor df and park df
combined_park_data_df = park_visitors_df_clean.merge(national_parks_df_clean, on="Park Name", how='left')

print(len(combined_park_data_df))
combined_park_data_df

#end Dan's code (modified for weather)

In [ ]:
weather_prep_data = combined_park_data_df[["Park Name", "Rank", "Visitors", "Percent Of Total", "Full Name", "Latitude", "Longitude", "Weather Info"]]
weather_data_prepped = weather_prep_data.copy()
weather_data_prepped["Lat/Lon"] = weather_prep_data["Latitude"].str.cat(weather_prep_data["Longitude"], sep = ", ")
weather_data_prepped.head()

In [ ]:
# API to Visual Crossing for weather data from summer of 2022
import urllib.parse

# Create list of results for all dates for each location
results = []

dates = ["2022-05-01",
        "2022-05-15",
        "2022-05-29",
        "2022-06-12",
        "2022-06-26",
        "2022-07-10",
        "2022-07-24",
        "2022-08-07",
        "2022-08-21",
        "2022-09-04",
        "2022-09-18"]

# set up a parameters dictionary
params = {
    "unitGroup":"us",
    "elements":"datetime,name,tempmax,tempmin,precip,cloudcover",
    "include":"days,obs",
    "key":VC_API_key,
    "contentType":"json"    
}
for park, row in weather_data_prepped.iterrows():
    lat = row["Latitude"]
    lon = row["Longitude"]
    for date in dates:
        # Set base URL
        # This url works:  url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/35.60116374%2C%20-83.50818326/2022-05-01/2022-05-01?unitGroup=us&elements=datetime%2Cname%2Ctempmax%2Ctempmin%2Cprecip%2Ccloudcover&include=days%2Cobs&key={VC_API_key}&contentType=json"
        base_url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{lat}%2C%20{lon}/{date}/{date}"
        query_string = urllib.parse.urlencode(params)  #from GPTChat
        
        # Build the URL using an f-string and get data
        url = f'{base_url}?{query_string}'
        # print(url)
        park_weather_data = requests.get(url).json()
        response = requests.get(url)
        if response.status_code == 200:
            park_weather_data = response.json()
            results.append(park_weather_data)
        else:
            print(f"Error: {response.status_code} - {response.text}")
        results.append(park_weather_data)


In [ ]:
# Create a dataframe with all the weather data

# Initialize empty lists for storing tempmin, tempmax, precip, and cloudcover values
lat_lon_ls = []
date_ls = []
tempmin_ls = []
tempmax_ls = []
precip_ls = []
cloudcover_ls = []

# Parse JSON and add our 4 values to their respective lists
for json_data in results:
    days_data = json_data["days"]
    if days_data:
        #Add lat,lon for eventual merge with park data.
        weather_lat = json_data["latitude"]
        weather_lon = json_data["longitude"]
        weather_lat_lon = f"{weather_lat}, {weather_lon}"
        lat_lon_ls.append(weather_lat_lon)
        day = days_data[0]
        obs_date = day["datetime"]
        date_ls.append(obs_date)
        tempmax = day["tempmax"]
        tempmax_ls.append(tempmax)
        tempmin = day["tempmin"]
        tempmin_ls.append(tempmin)
        precip = day["precip"]
        precip_ls.append(precip)
        cloudcover = day["cloudcover"]
        cloudcover_ls.append(cloudcover)


In [ ]:
# Create a DataFrame
weather_df = pd.DataFrame({
    "Lat/Lon" : lat_lon_ls,
    "Date": date_ls,
    "Temp Min": tempmin_ls,
    "Temp Max": tempmax_ls,
    "Precipitation": precip_ls,
    "Cloud Cover": cloudcover_ls
})

weather_df

In [ ]:
# Get average summer conditions for each park
avg_tempmin = weather_df.groupby("Lat/Lon") ["Temp Min"].mean()
avg_tempmax = weather_df.groupby("Lat/Lon") ["Temp Max"].mean()
avg_precip = weather_df.groupby("Lat/Lon") ["Precipitation"].mean()
avg_cloudcover = weather_df.groupby("Lat/Lon") ["Cloud Cover"].mean()

In [ ]:
# Create a df with the park averages
park_avgs = pd.DataFrame([avg_tempmin, avg_tempmax, avg_precip, avg_cloudcover])
park_avgs = park_avgs.transpose()

In [ ]:
# Merge park_avgs dataframe with weather_data on lat,lon and show
final_weather_data = weather_data_prepped.merge(park_avgs, how = "left", on = "Lat/Lon")
final_weather_data

In [ ]:
# Drop parks with missing temp data
# At lat/Lon:  
# 42.94065854, -122.1338414 = Crater Lake
# 61.4182147, -142.6028439 = Wrangell - St Elias
# And our known missing parks:  Sequoia, Kings Canyon, Redwood, and National Park of American Samoa

weather_minus_zeros = final_weather_data[final_weather_data["Temp Max"]!=0]
clean_weather_data = weather_minus_zeros.dropna(subset=['Temp Max'])
clean_weather_data.info()  #checking data type for visitors and how many NaN values are in Preciptation and Cloud Cover.

In [ ]:
# Scatterplot number of visitors vs avg temp max, pearson's r if it looks linear

visitors_by_thousand = clean_weather_data.copy()
visitors_by_thousand["Visitors"] = visitors_by_thousand["Visitors"].str.replace(",", "").astype(int)
visitors_by_thousand["Visitors (k)"] = visitors_by_thousand["Visitors"] / 1000
plt.scatter(visitors_by_thousand["Visitors (k)"], visitors_by_thousand["Temp Max"])
plt.title("NP Visitors by Avg Park Temperature")
plt.xlabel("Visitors in thousands")
plt.ylabel("Temperature (F)")
plt.show()

In [ ]:
#Linear regression between Avg Max Temp and Num of Visitors by Park

# Calculate the correlation coefficient
visitors = visitors_by_thousand["Visitors (k)"]
max_temp = visitors_by_thousand["Temp Max"]
print(f"The correlation coefficient between number of visitors and average maximum temperature is {round(st.pearsonr(visitors,max_temp)[0],2)}.")

pe_slope, pe_int, pe_r, pe_p, pe_std_err  = st.linregress(visitors, max_temp)
pe_fit = pe_slope * visitors + pe_int

plt.scatter(visitors_by_thousand["Visitors (k)"], visitors_by_thousand["Temp Max"])
plt.plot(visitors,pe_fit,"--",color="r")
plt.title("NP Visitors by Avg Park Temperature")
plt.xlabel("Visitors in thousands")
plt.ylabel("Temperature (F)")

plt.savefig("Images/Avg_Temp_Scatter.png") #dpi:  Higher number increases quality of picture AND size of file.  100 is default, 200 is double. Transparent applies to whether the background is clear or white.
plt.show()

In [ ]:
#Linear regression between Avg Min Temp and Num of Visitors by Park

# Calculate the correlation coefficient
visitors = visitors_by_thousand["Visitors (k)"]
min_temp = visitors_by_thousand["Temp Min"]
print(f"The correlation coefficient between number of visitors and average minimum temperature is {round(st.pearsonr(visitors,min_temp)[0],2)}.")

In [ ]:
#Linear regression between Avg Precipitation and Num of Visitors by Park

# Calculate the correlation coefficient
precip_df = visitors_by_thousand.dropna(subset = ["Precipitation"])
visitors = precip_df["Visitors (k)"]
precip = precip_df["Precipitation"]
print(f"The correlation coefficient between number of visitors and average precipitation is {round(st.pearsonr(visitors,precip)[0],2)}.")

In [ ]:
#Linear regression between Avg Cloud Cover and Num of Visitors by Park

# Calculate the correlation coefficient
cloudcover_df = visitors_by_thousand.dropna(subset = ["Cloud Cover"])
visitors = cloudcover_df["Visitors (k)"]
cloudcover = cloudcover_df["Cloud Cover"]
print(f"The correlation coefficient between number of visitors and average cloud cover is {round(st.pearsonr(visitors,cloudcover)[0],2)}.")

In [ ]:
# Put parks in bins by avg maxtemp ranges
park_bins = clean_weather_data.copy()
park_bins["Visitors"] = park_bins["Visitors"].str.replace(",", "").astype(int)
park_bins["Visitors (k)"] = park_bins["Visitors"] / 1000
bins = [0, 60, 70, 80, 90, 100]
labels = ['<60 degrees', '60-70 degrees', '70-80 degrees', '80-90 degrees', '90-100 degrees']
park_bins['Avg Temp Range'] = pd.cut(x = park_bins['Temp Max'], bins = bins, labels = labels, include_lowest = True)
park_bins.head()

In [ ]:
# Are more high visitor parks in any bin?
# Create a box plot with temperature range bins.

temp_ranges = ['<60 degrees', '60-70 degrees', '70-80 degrees', '80-90 degrees', '90-100 degrees']

# Create empty list to fill with grouped visitor data
visitors_by_range = []

# Create list of visitors for each temperature range.
for temp in temp_ranges: 
    range_group = park_bins.loc[park_bins["Avg Temp Range"] == temp, "Visitors (k)"]
    visitors_by_range.append(range_group)
    quartiles = range_group.quantile([.25,.5,.75])
    lowerq = quartiles[0.25]
    upperq = quartiles[0.75]
    iqr = upperq-lowerq
    lower_bound = lowerq - (1.5*iqr)
    upper_bound = upperq + (1.5*iqr)
    
plt.boxplot(visitors_by_range, labels = temp_ranges)
plt.xticks(rotation = 45)
plt.title("Visitors to Parks by Temperature Ranges")
plt.xlabel("Average Temperature Range (F)")
plt.ylabel("Annual Visitors (k)")

plt.savefig("Images/Temperature_Range_Box_Plot.png", bbox_inches='tight') #dpi:  Higher number increases quality of picture AND size of file.  100 is default, 200 is double. Transparent applies to whether the background is clear or white.
plt.show()

In [ ]:
# Statistical Testing
# GPTChat suggested that the best statistical test for numerical data in 5 categories is the Kruskal-Wallis test rather than any we learned in class.
# The next code is how ChatGPT recommended splitting visitors_by_range (the list of series of visitors by temperature range bins) into 5 separate lists to use for the test.

import string

series_list = visitors_by_range

group_dict = {}
groups = string.ascii_uppercase[:len(series_list)]  # A, B, C, D, E

for group, series in zip(groups, series_list):   
    group_dict[group] = series.tolist()

group_A = group_dict['A']
group_B = group_dict['B']
group_C = group_dict['C']
group_D = group_dict['D']
group_E = group_dict['E']

# Perform Kruskal-Wallis test
statistic, p_value = st.kruskal(group_A, group_B, group_C, group_D, group_E)
print(p_value)